# How to establish a strategy with really strong 57% returns

Asset class trend-following is a strategy that takes uses momentum and moving averages. It identifies periods of potential outperformance to minimize volatility and drawdowns.

It was first introduced by Meb Faber and widely accepted in academia.

Professionals apply asset class trend-following by incorporating momentum filters for different asset classes. They use it to stay invested in asset classes that are performing well while avoiding those with higher risk.

By understanding and implementing this strategy, you can manage risk-adjusted returns. It helps achieve a balance of equity-like returns with bond-like volatility and drawdowns.

In [1]:
# We’ll use the Zipline backtesting framework to assess the strategy. PyFolio is great for risk and performance analysis.

import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from zipline import run_algorithm
from zipline.api import (
    attach_pipeline,
    date_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
    symbol,
    time_rules,
    get_open_orders,
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

import pyfolio as pf

## Set up the strategy code

Start with building the function that’s called at the beginning of the backtest. The strategy logic happens in the next function.

In [2]:
def initialize(context):
    context.symbols = [
        symbol("SPY"),
        symbol("EFA"),
        symbol("IEF"),
        symbol("VNQ"),
        symbol("GSG"),
    ]
    context.sma = {}
    context.period = 10 * 21

    for asset in context.symbols: 
        context.sma[asset] = SimpleMovingAverage(
            inputs=[USEquityPricing.close],
            window_length=context.period
        )

    schedule_function(
        func=rebalance,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open(minutes=1),
    )

    context.set_commission(
        commission.PerShare(cost=0.01, min_trade_cost=1.00)
    )
    context.set_slippage(slippage.VolumeShareSlippage())
# The logic is straightforward: For each ETF representing an asset class, calculate the 10-month simple moving average using the closing price.

# This is a long-term strategy that rebalances at the market open on the first day of the month. Zipline’s schedule function makes it simple to set this up. Note we schedule a function called rebalance to run on this schedule. You’ll create it next.

# Finally, include realistic commission and slippage models.